In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip uninstall -y httpx
!pip install httpx==0.23.0
!pip install gradio==3.16.0
!pip install googletrans==4.0.0-rc1
!pip install transformers==4.30.2
!pip install torch==2.0.1
!pip uninstall -y tensorflow keras tensorflow-text
!pip install tensorflow==2.11 keras==2.11 transformers==4.30.2
!pip install torch==2.0.1 sentencepiece
!pip install googletrans==4.0.0-rc1

In [ ]:
import gradio as gr
from googletrans import Translator
from transformers import pipeline

In [ ]:
# Initialize translator and summarization model
translator = Translator()
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


In [ ]:
# # Initialize translator and summarization model
# translator = Translator()
# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to translate text
def translate_text(text, target_lang):
    translation = translator.translate(text, dest=target_lang)
    return translation.text

# Function to summarize and translate back
def summarize_and_translate(input_text):
    # Translate to English
    translated_text = translate_text(input_text, "en")
    
    # Summarize the translated text
    summary = summarizer(translated_text, max_length=130, min_length=30, do_sample=False)
    summarized_text = summary[0]["summary_text"]
    
    # Translate summary back to Persian
    summarized_back = translate_text(summarized_text, "fa")
    
    return {
        "translated_text": translated_text,
        "summary_text": summarized_text,
        "translated_back": summarized_back
    }

# Create Gradio interface
def run_interface():
    with gr.Blocks() as demo:
        gr.Markdown("""# متن خود را وارد کنید و عملیات ترجمه و خلاصه‌سازی را انجام دهید.""")
        
        with gr.Row():
            input_text = gr.Textbox(label="متن ورودی (فارسی)", placeholder="متن خود را اینجا وارد کنید.", lines=5)
        
        with gr.Row():
            output_translated = gr.Textbox(label="متن ترجمه شده (انگلیسی)", lines=5)
            output_summary = gr.Textbox(label="خلاصه (انگلیسی)", lines=5)
            output_back = gr.Textbox(label="متن خلاصه‌شده ترجمه‌شده (فارسی)", lines=5)
        
        def process(input_text):
            results = summarize_and_translate(input_text)
            return results["translated_text"], results["summary_text"], results["translated_back"]
        
        submit_button = gr.Button("اجرا")
        submit_button.click(process, inputs=[input_text], outputs=[output_translated, output_summary, output_back])

    demo.launch()

if __name__ == "__main__":
    run_interface()
